# Testing of features in polysaccharide2

In [1]:
# Supressing annoying warnings (!must be done first!)
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

# Logging
from polysaccharide2.genutils.logutils.IOHandlers import LOG_FORMATTER

import logging
logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    # force=True
)
LOGGER = logging.Logger(__name__)

# General
import re, json
from pathlib import Path
from shutil import copyfile

import numpy as np

# Logging
from rich.progress import Progress, track
import logging

# Chemistry
from openmm.unit import nanometer, angstrom
from openff.toolkit import Topology, Molecule, ForceField
from openff.units import unit as offunit

from openff.interchange import Interchange
from openff.interchange.components import _packmol as packmol

from rdkit import Chem
import openeye

# Custom
import polysaccharide2 as ps2
from polysaccharide2.genutils.decorators.functional import allow_string_paths, allow_pathlib_paths, optional_in_place

/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


# OpenFF parameterization testing

## Testing load using from_pdb and monomers

In [ ]:
from polysaccharide2.openfftools import TKREGS
from polysaccharide2.openfftools import topIO
from polysaccharide2.openfftools.topinfo import get_largest_offmol
from polysaccharide2.residues.partition import partition
from polysaccharide2.monomers.repr import MonomerGroup

# pdb_dir  = Path('polymer_examples/compatible_pdbs/simple_polymers')
# mono_dir = Path('polymer_examples/monomer_generation/json_files/')

pdb_sub = 'simple_polymers'
pdb_dir  = Path(f'pdb_test_cleaned/pdbs/{pdb_sub}')
mono_dir = Path(f'pdb_test_cleaned/monos/{pdb_sub}')

mol_name = 'polyvinylchloride'
# mol_name = 'PEO_PLGA'
# mol_name = 'paam_modified'
# pdb_sub = 'proteins'
# mol_name = '6cww'

pdb = pdb_dir / f'{mol_name}.pdb'
mono = mono_dir / f'{mol_name}.json'
assert(pdb.exists())
assert(mono.exists())

monogrp = MonomerGroup.from_file(mono)
rdmol = Chem.MolFromPDBFile(str(pdb))
offtop = Topology.from_pdb(pdb, _custom_substructures=monogrp.monomers, toolkit_registry=TKREGS['The RDKit'])
was_partitioned = partition(offtop)
print(was_partitioned)

# assign properties to Molecule
offmol = get_largest_offmol(offtop)
offmol.name = mol_name
offmol.properties['solvent'] = None
offmol.properties['charge_method'] = None

# save partitioned Topology
sdf_dir = Path('sdf_test')
sdf_dir.mkdir(exist_ok=True)
mol_path = sdf_dir / f'{mol_name}.sdf'
topIO.topology_to_sdf(mol_path, offtop=offtop, toolkit_registry=TKREGS['The RDKit'])

## Partial charge assignment

In [ ]:
from polysaccharide2.openfftools import TKREGS
from polysaccharide2.openfftools import topIO
from polysaccharide2.residues.rescharge import application, calculation

base_charge_method = 'AM1-BCC-ELF10'

# assign charges with default methods
charged_mols = {}
for charge_method, ChargerType in application.MolCharger.subclass_registry.items():
    chgr = ChargerType()
    cmol = charged_mols[charge_method] = chgr.charge_molecule(offmol, in_place=False)

# generate library charges and charge by residue
res_chg = calculation.compute_residue_charges(charged_mols[base_charge_method], monogrp)
res_chg.to_file(sdf_dir / f'{mol_name}_residue_charges.json')

offmol_avg = application.apply_residue_charges(offmol, res_chg, in_place=False)
offmol_avg.properties['charge_method'] = 'RCT-averaged'
charged_mols['RCT-averaged'] = offmol_avg

# saving charged molecules to SDF files
for charge_method, cmol in charged_mols.items():
    topIO.topology_to_sdf(sdf_dir / f'{mol_name}_{charge_method}.sdf', cmol.to_topology())

## Testing building and RCT

In [7]:
from pathlib import Path

from polysaccharide2.genutils.decorators.functional import allow_string_paths
from polysaccharide2.monomers.repr import MonomerGroup
from polysaccharide2.polymers import estimation, building
from polysaccharide2.polymers.exceptions import MorphologyError

from polysaccharide2.openfftools import TKREGS, topIO, topinfo
from polysaccharide2.openfftools.pcharge import MolCharger

from polysaccharide2.residues.partition import partition
from polysaccharide2.residues.rescharge.rctypes import ChargesByResidue
from polysaccharide2.residues.rescharge.calculation import compute_residue_charges


@allow_string_paths
def rct_protocol(pdb_path : Path, monogroup : MonomerGroup, term_group_orient : dict[str, str], N : int, charger : MolCharger, delete_pdb : bool=True, save_sdf : bool=False) -> ChargesByResidue:
    '''
    Generates library charges for a monomer group given terminal group head-tail orientations and a maximum chain length
    If delete_pdb=True, will remove the working pdb path after charges are generated
    as currently implemented, only supports monomer groups which constitute a linear homopolymer
    '''
    mol_name = pdb_path.stem
    if not monogroup.is_linear:
        raise MorphologyError('RCT currently only supports linear homopolymers')
    
    DOP = estimation.estimate_DOP_lower(monogroup, max_chain_len=N)
    chain = building.build_linear_polymer(monogrp, DOP, term_orient=term_group_orient)

    building.mbmol_to_openmm_pdb(pdb_path, chain) # output PDB file to disc 
    offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers, toolkit_registry=TKREGS['The RDKit']) # load custom substructures - raises error if PDB has issues
    if delete_pdb:
        pdb_path.unlink()
        
    was_partitioned = partition(offtop) 
    assert(was_partitioned)
    offmol = topinfo.get_largest_offmol(offtop)
    offmol.name = mol_name

    cmol = charger.charge_molecule(offmol, in_place=False)
    if save_sdf:
        topIO.topology_to_sdf(pdb_path.with_name(f'{pdb_path.stem}.sdf'), cmol.to_topology())
    res_chgs = compute_residue_charges(cmol, monogroup)

    return res_chgs

In [10]:
from polysaccharide2.openfftools.pcharge import MolCharger
from polysaccharide2.residues.rescharge.interface import LibraryCharger

# building parameters
outdir = Path('RCT_test')
outdir.mkdir(exist_ok=True)

DOP = 8
N = 100
chgr = MolCharger.subclass_registry['Espaloma-AM1-BCC']()
delete_pdb : bool=True
save_sdf : bool=True

term_groups = { 
    'peg_modified' : {
        'peg_TERM2' : 'head',
        'peg_TERM3' : 'tail',
    },
    'paam_modified' : {
        'paam_TERM2' : 'head',
        'paam_TERM3' : 'tail',
    },
    'pnipam_modified' : {
        'pnipam_TERM2' : 'head',
        'pnipam_TERM3' : 'tail',
    },
}

# defining paths
mol_category = 'simple_polymers'
pdb_dir  = Path(f'pdb_test_cleaned/pdbs/{mol_category}')
mono_dir = Path(f'pdb_test_cleaned/monos/{mol_category}')

# estimation and building loop
for mol_name, term_group_orient in term_groups.items():
    pdb_temp   = outdir / f'{mol_name}_redux.pdb' # NOTe : new PDB path, not an existing one
    # pdb_temp = 'polymer.pdb'
    mono_path = mono_dir / f'{mol_name}.json'
    assert(mono_path.exists())

    monogrp = MonomerGroup.from_file(mono_path)
    lib_chgs = rct_protocol(pdb_temp, monogrp, term_group_orient, N, charger=chgr, delete_pdb=delete_pdb, save_sdf=save_sdf)
    lib_chgs.to_file(outdir / f'{mol_name}_residue_charges.json')

    # PHASE 2: applying library charges to ful-size mol
    pdb_path = pdb_dir / f'{mol_name}.pdb' # path to the true, full-size PDB 
    offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers, toolkit_registry=TKREGS['The RDKit'])
    was_partitioned = partition(offtop) 
    assert(was_partitioned)
    fullmol = topinfo.get_largest_offmol(offtop)
    
    lib_chgr = LibraryCharger(lib_chgs)
    rctmol = lib_chgr.charge_molecule(fullmol, in_place=False)
    topIO.topology_to_sdf(outdir / f'{mol_name}_{lib_chgr.CHARGING_METHOD}.sdf', rctmol.to_topology())

2023-10-09 20:03:00.281 [INFO    :        building:line 82  ] - Building linear polymer chain with 12 monomers (93 atoms)


2023-10-09 20:03:01.690 [INFO    :         pcharge:line 34  ] - Assigning partial charges via the "Espaloma-AM1-BCC" method
/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/espaloma_charge/openff_wrapper.py:112: IncorrectNumConformersWarning: Molecule 'Molecule with name 'peg_modified_redux' and SMILES '[H]C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])[H]'' has 1 conformers, but charge method 'espaloma-am1bcc' expects exactly 0.
  self._check_n_conformers(
2023-10-09 20:03:01.825 [INFO    :         pcharge:line 37  ] - Successfully assigned "Espaloma-AM1-BCC" charges
2023-10-09 20:03:01.873 [INFO    :     calculation:line 32  ] - Selected representative residue groups
2023-10-09 20:03:01.875 [INFO    :     calculat

## Solvation of Topologies

In [15]:
from openmm.unit import gram, centimeter, nanometer, mole
from polysaccharide2.openfftools.solvation.packing import pack_topology_with_solvent
from polysaccharide2.openfftools.solvation.solvents import water_TIP3P

# PARAMETERS
targ_box_vecs = 6.0 * np.ones(3) * nanometer
density = 0.997 * (gram / centimeter**3)
exclusion = 1.3 * nanometer
solvent = water_TIP3P

solv_top = pack_topology_with_solvent(rctmol.to_topology(), solvent, box_vecs=targ_box_vecs, density=density, exclusion=exclusion)
topIO.topology_to_sdf(outdir / f'{mol_name}_{lib_chgr.CHARGING_METHOD}_solv_{solvent.name}.sdf', solv_top)

In [ ]:
from polysaccharide2.openmmtools.parameters import SimulationParameters
from copy import deepcopy

def conformer_anneal(offtop : Topology, num_confs : int, anneal_params : SimulationParameters, forcefield : ForceField) -> Topology:
    '''Takes a Topology, a number of confomrers to generate, and parameters describing an annealing simulation 
    Returns a corresponding Topology with target unmber of new conformers, each taken as the final snapshot of each simulation'''
    conf_top = deepcopy(offtop)
    inc = Interchange.from_smirnoff(offtop, forcefield)

# OpenMM I/O and simulation interfaces

## Initialize OpenMM sim + files

In [ ]:
from openmm.unit import nanosecond, picosecond, femtosecond
from openmm.unit import kelvin, atmosphere, nanometer, centimeter
from openmm.unit import gram, mole, liter
from polysaccharide2.openmmtools import parameters, serialization, preparation

from polysaccharide2.openfftools import topIO, topinfo
from polysaccharide2.openfftools.omminter import openff_topology_to_openmm
from polysaccharide2.openfftools.solvation import packing


# input parameters
sdf_dir = Path('sdf_test')
sdf_path = sdf_dir / 'polyvinylchloride_AM1-BCC-ELF10.sdf'

box_dims = 4.0 * np.ones(3) * nanometer
density = 0.997 * gram/centimeter**3
exclusion = 1.0 * nanometer
ff_name = 'openff-2.0.0'

# define paths
prefix = 'pvc'
save_path = Path('openmm_test')
save_path.mkdir(exist_ok=True)

# initialize and integrate simulation
offtop = topIO.topology_from_sdf(sdf_path)
water  = Molecule.from_file('water_files/water_TIP3P.sdf')
packtop = packing.pack_topology_with_solvent(offtop, solvent=water, box_vecs=box_dims, density=density, exclusion=exclusion)

ommobjs = openff_topology_to_openmm(offtop, forcefield=ff_name, box_vecs=box_dims)

In [ ]:
from polysaccharide2.genutils.logutils.IOHandlers import MSFHandlerFlex, get_logger_registry, get_active_loggers

p = Path('sim_param_sets')
schedule = {
    'anneal' : parameters.SimulationParameters.from_file(p / 'anneal_params.json'),
    'equil' : parameters.SimulationParameters.from_file(p / 'equilibration_params.json'),
    'prod' : parameters.SimulationParameters.from_file(p / 'production_lite_params.json'),
}
for params in schedule.values():
    if params.thermo_params.ensemble == 'NPT':
        params.thermo_params.ensemble = 'NVT'

    if params.integ_params.total_time >= 1*nanosecond:
        params.integ_params.total_time = 100*picosecond

with MSFHandlerFlex(save_path, proc_name='sim_schedule', loggers='all') as log_handler:
    preparation.run_simulation_schedule(save_path, schedule, *ommobjs)

In [ ]:
param_top_path = Path(...)

# Vacuum anneal

# Solvation

# Equilibration

# Production

### Copying coordinate to single chain after anneal

In [ ]:
from copy import deepcopy

final_state = ommsim.context.getState(getPositions=True)
final_pos = final_state.getPositions(asNumpy=True)

new_mol = deepcopy(offmol)
new_mol.conformers[0] = openmm_to_openff(final_pos.in_units_of(angstrom)) # convert to correct units in the OpenFF format

In [ ]:
topIO.topology_to_sdf('orig.sdf', offmol.to_topology())
topIO.topology_to_sdf('shifted.sdf', new_mol.to_topology())

# Playing with ratios

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable, ClassVar, TypeVar
from math import gcd
from numbers import Number


N = TypeVar('N')
def sgnmag(num : N) -> tuple[bool, N]:
    '''Returns the sign and magnitude of a numeric-like value'''
    return num < 0, abs(num)


@dataclass(repr=False)
class Ratio:
    '''For representing fractional ratios between two objects'''
    num   : Any
    denom : Any

    # REPRESENTATION
    def __repr__(self) -> str:
        return f'{self.num}/{self.denom}'
    
    def to_latex(self) -> str:
        '''Return latex-compatible string which represent fraction'''
        return rf'\frac{{{self.num}}}{{{self.denom}}}'

    # RELATIONS
    @property
    def reciprocal(self) -> 'Ratio':
        '''Return the reciprocal of a ration'''
        return self.__class__(self.denom, self.num)


@dataclass(repr=False)
class Rational(Ratio):
    '''For representing ratios of integers'''
    num   : int
    denom : int

    # REDUCTION
    autoreduce : ClassVar[bool]=False
    
    def __post_init__(self) -> None:
        if self.__class__.autoreduce:
            self.reduce()

    def reduce(self) -> None:
        '''Reduce numerator and denominator by greatest common factor'''
        _gcd = gcd(self.num, self.denom)
        self.num=int(self.num / _gcd)
        self.denom=int(self.denom / _gcd)
    simplify = reduce # alias for convenience

    @property
    def reduced(self) -> 'Rational':
        '''Return reduced Rational equivalent to the current rational (does not modify in-place)'''
        new_rat = self.__class__(self.num, self.denom)
        new_rat.reduce()

        return new_rat
    simplifed = reduced # alias for convenience
    
    def as_proper(self) -> tuple[int, 'Rational']:
        '''Returns the integer and proper fractional component of a ratio'''
        integ, remain = divmod(self.num, self.denom)
        return integ, self.__class__(remain, self.denom)
    
    # ARITHMETIC
    def __add__(self, other : 'Rational') -> 'Rational':
        '''Sum of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) + (self.denom * other.num),
            denom=(self.denom * other.denom)
        )
    
    def __sub__(self, other : 'Rational') -> 'Rational':
        '''Difference of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) - (self.denom * other.num),
            denom=(self.denom * other.denom)
        )

    def __mul__(self, other : 'Rational') -> 'Rational':
        '''Product of two Rationals'''
        return self.__class__(
            num=self.num * other.num,
            denom=self.denom * other.denom
        )

    def __div__(self, other : 'Rational') -> 'Rational':
        '''Quotient of two Rationals'''
        return self.__class__(
            num=self.num * other.denom,
            denom=self.denom * other.num
        )
    
    def __pow__(self, power : float) -> 'Rational':
        '''Exponentiates a ratio'''
        return self.__class__(
            num=self.num**power,
            denom=self.denom**power
        )

In [ ]:
p = Rational(3, 6)
q = Rational(4, 12)

print(p, p.reciprocal, p.reduced, p+q)

In [ ]:
Rational.autoreduce = False

In [ ]:
import numpy as np
from numbers import Number

for val in (4, 4.0, 4+0j, np.pi, '4', [4], False, 'sgdfg'):
    print(val, type(val), isinstance(val, Number))

In [ ]:
from fractions import Fraction